# Heavy Compute + Rotation Optimization
Scaling up the C++ optimizer and adding a Python-based rotation optimization step.

In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

print(f"Current working directory: {os.getcwd()}")

# Setup paths
DATA_DIR = '/home/data'
START_SUBMISSION = 'submission_start.csv'
CURRENT_SUBMISSION = 'submission.csv'

# Ensure binaries are executable
subprocess.run(['chmod', '+x', './tree_packer'], check=True)
subprocess.run(['chmod', '+x', './bp'], check=True)

# ----------------------------------------------------------------
# Geometry & Validation Classes
# ----------------------------------------------------------------
getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x).strip('s'))
        self.center_y = Decimal(str(center_y).strip('s'))
        self.angle = Decimal(str(angle).strip('s'))
        
        # Tree definition
        trunk_w = Decimal('0.15'); trunk_h = Decimal('0.2')
        base_w = Decimal('0.7'); mid_w = Decimal('0.4'); top_w = Decimal('0.25')
        tip_y = Decimal('0.8'); tier_1_y = Decimal('0.5'); tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0'); trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(self.center_x, self.center_y, self.angle)

def load_configuration_from_df(n: int, df: pd.DataFrame) -> list[ChristmasTree]:
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        trees.append(ChristmasTree(row["x"], row["y"], row["deg"]))
    return trees

def get_score(trees: list[ChristmasTree], n: int) -> float:
    if not trees: return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

def has_overlap(trees: list[ChristmasTree]) -> bool:
    if len(trees) <= 1: return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i: continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

# ----------------------------------------------------------------
# Rotation Optimization Logic
# ----------------------------------------------------------------
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation_for_group(trees):
    if not trees: return trees, 0.0
    
    # Collect all points
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    # Convex Hull
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np # Fallback if hull fails (e.g. collinear)

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    # Optimize angle
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.0, 90.0), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    if found_side < initial_side:
        best_angle_deg = found_angle_deg
    else:
        best_angle_deg = 0.0
        
    # Apply rotation
    if abs(best_angle_deg) > 1e-9:
        # Rotate all trees around the center of the group
        # Center of rotation: center of the bounding box
        min_x, min_y = points_np.min(axis=0)
        max_x, max_y = points_np.max(axis=0)
        rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
        
        angle_rad = np.radians(best_angle_deg)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        rot_matrix = np.array([[c, -s], [s, c]])
        
        new_trees = []
        for tree in trees:
            # Rotate center
            p = np.array([float(tree.center_x), float(tree.center_y)])
            shifted = p - rotation_center
            rotated_p = shifted.dot(rot_matrix.T) + rotation_center
            
            # New angle
            new_angle = float(tree.angle) + best_angle_deg
            new_trees.append(ChristmasTree(rotated_p[0], rotated_p[1], new_angle))
        return new_trees, best_angle_deg
    else:
        return trees, 0.0

def run_rotation_optimization(input_csv, output_csv):
    print(f"Running Rotation Optimization on {input_csv}...")
    df = pd.read_csv(input_csv)
    
    new_rows = []
    
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        if not trees: continue
        
        # Optimize
        optimized_trees, angle_change = optimize_rotation_for_group(trees)
        
        for i, tree in enumerate(optimized_trees):
            new_rows.append({
                'id': f'{n:03d}_{i}',
                'x': f's{tree.center_x}',
                'y': f's{tree.center_y}',
                'deg': f's{tree.angle}'
            })
            
    new_df = pd.DataFrame(new_rows)
    # Ensure column order
    new_df = new_df[['id', 'x', 'y', 'deg']]
    new_df.to_csv(output_csv, index=False)
    print(f"Rotation Optimization complete. Saved to {output_csv}")

# ----------------------------------------------------------------
# Main Pipeline
# ----------------------------------------------------------------
class CppOptimizer:
    def __init__(self):
        pass
        
    def run_packer(self, input_file, output_file, iterations=200000, restarts=32):
        print(f"Running tree_packer: n={iterations}, r={restarts}...")
        subprocess.run(['./tree_packer', '-i', input_file, '-o', output_file, '-n', str(iterations), '-r', str(restarts)], check=True)
        
    def run_bp(self, input_file, output_file):
        print("Running Backward Propagation...")
        subprocess.run(['./bp', input_file, output_file], check=True)

# Execution
model = CppOptimizer()

# 1. Initialize
if os.path.exists(START_SUBMISSION):
    print(f"Starting from {START_SUBMISSION}")
    subprocess.run(['cp', START_SUBMISSION, CURRENT_SUBMISSION], check=True)
else:
    print("Starting from sample_submission.csv")
    subprocess.run(['cp', os.path.join(DATA_DIR, 'sample_submission.csv'), CURRENT_SUBMISSION], check=True)

# 2. Heavy Pack (20k)
model.run_packer(CURRENT_SUBMISSION, 'submission_packed.csv', iterations=20000, restarts=32)

# 3. Rotate
run_rotation_optimization('submission_packed.csv', 'submission_rotated.csv')

# 4. BP
model.run_bp('submission_rotated.csv', 'submission_bp.csv')

# 5. Final Pack (10k)
model.run_packer('submission_bp.csv', CURRENT_SUBMISSION, iterations=10000, restarts=16)

print("Pipeline finished.")

Current working directory: /home/code
Starting from sample_submission.csv
Running tree_packer: n=20000, r=32...


Tree Packer v21 - ENHANCED (26 threads)
NEW: Swap moves, multi-angle restarts, higher SA temperature
Iterations: 20000, Restarts: 32
Processing all n=1..200 concurrently
Loading submission.csv...
Loaded 200 configs
Initial: 173.652299

Phase 1: Parallel optimization...

n=  1: 1.000000 -> 0.661250 (33.8750%)


n=  2: 0.733589 -> 0.451027 (38.5177%)


n=  3: 0.930301 -> 0.434981 (53.2430%)


n=  4: 1.039642 -> 0.437907 (57.8791%)


...

n= 45: 0.625747 -> 0.603853 (from n=46 removal, 3.4989%)


n= 38: 0.593930 -> 0.577383 (from n=39 removal, 2.7861%)


n= 37: 0.614415 -> 0.578141 (from n=38 removal, 5.9039%)


n= 36: 0.625087 -> 0.577001 (from n=37 removal, 7.6927%)
n= 35: 0.613799 -> 0.589609 (from n=36 removal, 3.9411%)


n= 20: 0.570391 -> 0.565532 (from n=21 removal, 0.8519%)
Pass 1 complete, continuing...

Initial: 124.916439
Final:   121.037795
Improve: 3.878643 (3.1050%)
Phase 1 improved: 196 configs
Phase 2 back-prop improved: 92 configs
Time:    782.7s (with 26 threads)
Saved submission.csv
Pipeline finished.


In [2]:
# Validation
def score_and_validate_submission(file_path: str, max_n: int = 200) -> dict:
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        return {"status": "FAILED", "error": "File Not Found"}
    
    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.12f})")
    
    if failed_overlap_n:
        status = "FAILED (Overlaps)"
    else:
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N):   {total_score:.14f}**")
    return {"status": status, "total_score": total_score, "failed_overlap_n": failed_overlap_n}

result = score_and_validate_submission(CURRENT_SUBMISSION)
print(f"Final Result: {result}")


--- Scoring and Validation: submission.csv (N=1 to 200) ---


**Total Submission Score (Σ S²/N):   121.03779543040088**
Final Result: {'status': 'SUCCESS', 'total_score': 121.03779543040088, 'failed_overlap_n': []}
